In [311]:
#Import Lib
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score, f1_score

from tensorflow.keras.layers import Dense, LSTM, RNN, Embedding, Dropout
from tensorflow.keras.models import Sequential

from keras.backend import clear_session

import os

In [312]:
df = pd.read_csv(r'D:/Users/Desktop/Scientific-research/Deep learning/Data/Android-malware/drebin-215-dataset-5560malware-9476-benign.csv')

C:\Users\LEGION\AppData\Local\Temp\ipykernel_14596\2343203276.py:1: DtypeWarning: Columns (92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:/Users/Desktop/Scientific-research/Deep learning/Data/Android-malware/drebin-215-dataset-5560malware-9476-benign.csv')


In [313]:
df.head()

,transact,onServiceConnected,bindService,attachInterface,ServiceConnection,android.os.Binder,SEND_SMS,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.getMethods,Ljava.lang.Class.cast,...,READ_CONTACTS,DEVICE_POWER,HARDWARE_TEST,ACCESS_WIFI_STATE,WRITE_EXTERNAL_STORAGE,ACCESS_FINE_LOCATION,SET_WALLPAPER_HINTS,SET_PREFERRED_APPLICATIONS,WRITE_SECURE_SETTINGS,class
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,S
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,S
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,S
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,1,1,0,0,0,S
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,S


In [314]:
df = df.dropna()

In [315]:
#Pretrain dataset
def preprocess(dataframe):
    dataframe.loc[dataframe['class'] == "B", "class"] = 0
    dataframe.loc[dataframe['class'] == "S", "class"] = 1

    return dataframe

In [316]:
scaled_train = preprocess(df)

X = scaled_train.drop(['class'], axis=1).values
y = scaled_train['class'].values

# Replace '?' with NaN
df.replace('?', np.nan, inplace=True)

# Convert columns to numeric data type
df = df.apply(pd.to_numeric)

# Replace NaN values with mean of column
df.fillna(df.mean(), inplace=True)

# Convert labels to integers
df['class'] = df['class'].astype('category').cat.codes

# Convert labels to integers
y = y.astype('int')

#Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [317]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(12028, 215)
(3008, 215)
(12028,)
(3008,)


In [318]:
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))

In [319]:
model = Sequential()
model.add(LSTM(units=32, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.3))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 32)                4352      
                                                                 
 dropout_18 (Dropout)        (None, 32)                0         
                                                                 
 dense_24 (Dense)            (None, 1)                 33        
                                                                 
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________


In [320]:
# Compile models
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit model
history = model.fit(X_train.reshape((X_train.shape[0], X_train.shape[1], 1)), y_train, epochs=10, batch_size=32,
                    validation_data=(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)), y_test))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).